In [1]:
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt
import datetime as dt
from zigzag_dag26 import zigzag
import talib
from tqdm import tqdm
import colorama as col

In [57]:
%%time
##### Chargement de la base
base = pd.read_csv('Base_Clean/EURUSDm5.csv')

##### On fixe la date en index sous forme de Timestamp
base.set_index(pd.to_datetime(base.Date),drop=True,inplace=True)

###### On drop les colonnes inutiles
base = base.drop(['Date','Total'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
base['WE'] = np.where(((base.index.weekday == 5) | (base.index.weekday == 6)),None,base.index.weekday)
base = base.dropna()
base = base.drop(['WE'],axis=1)


CPU times: user 2.06 s, sys: 241 ms, total: 2.3 s
Wall time: 2.31 s


In [58]:
base.index[0],base.index[-1]

(Timestamp('2010-11-23 00:00:00'), Timestamp('2020-11-23 15:40:00'))

In [153]:
%%time
_year_bottom = pd.to_datetime('2020-11-01 16:00:00')
_year_top = pd.to_datetime('2020-11-23 16:00:00') #2020-11-23 16:00:00

df = base[(base.index>=_year_bottom)&(base.index<=_year_top)]

df_input = df.copy()

df_bt = df.copy()

CPU times: user 5.22 ms, sys: 2.63 ms, total: 7.84 ms
Wall time: 6.25 ms


In [154]:
%%time
from fibostoch import Fibo_Stoch
df = Fibo_Stoch(df)

100%|██████████| 4508/4508 [00:03<00:00, 1193.33it/s]CPU times: user 8.26 s, sys: 512 ms, total: 8.77 s
Wall time: 8.81 s



In [155]:
# < 10 > 10 > 90 < 90
df['Signal'] = np.where(\
    ((df.FiboStoch < 10)&(df.FiboStoch.shift(1) > 10)),1,\
        np.where(\
            ((df.FiboStoch > 90)&(df.FiboStoch.shift(1) < 90)),-1,0)
    )


In [156]:
########################
####### BT PRECIS ######
########################

In [157]:
from bt import bt

In [164]:
_nb_bougie_exit = 60
_trigger_reengage = 0
_trigger_target = 0
_trigger_invers = 0
_trigger_sl = 0
_verbose = 0
_cash_ini = 200000
_rate = 1
x = 'EUR/USD'
_target = 0.002
_exposure = 10
_size = 50000
_sl = 0.0007


In [165]:
bt(df,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

Librairies imported

Début des opérations horodatée à 2020-12-08 19:25:54.991273

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 4388/4388 [00:00<00:00, 7085.42it/s]
 Pour le ticker  EUR/USD 
 
Les gains faramineux s'élèvent à : $  406.0 !. En  50   transactions.
 
Nbre de winners : 25 
 
Nbre de loosers : 24 
Temps d'excution du BT 0:00:06.854223
 EUR/USD  results 
 Tested Period 2020-11-01 16:00:00  à 2020-11-23 16:00:00 
 Total Number of trades 50 
Started Cash : 50000
P&L  in currency:  406.0$ 
P&L in %:  0.2% 
Average trade duration 60.0
# Winners  25.0
# Loosers  24.0
Cumulated gains 2632.0
Cumulated losses -2226.0
 PROFIT FACTOR :  1.18 
 Winners Ratio : 51.02 % 
Average Winners 52.64
Average Loosers -44.52
Average pnl 8.12
% Average Winners 0.03
% Average Loosers -0.02
% Average pnl 0.0
Number of opened trades 50
Number of closed trades 49
Max Exposure 1 x  50000 =  50000 $


()

In [ ]:
###################################
##### FiboStoch + Retracement #####
###################################

In [144]:
######################
##### VANILLA BT #####
######################

In [145]:
from vanilla_bt import *

In [17]:
bt_tpsl(df,0.01,0.007)

fermeture short en l'air
Winner Ratio : 47.37 %
Profit Factor :  1.27 % 
 Période du 2020-01-23 16:00:00 au  2020-11-23 16:00:00 
Nombre de trades : 57
Nombre de posistions non fermées 0
Captital initial : 250000
Taille des positions : 50000
Gain :  3476.749999999985 
Capital Final : 253476.74999999997


()

In [8]:
################
##### GRID #####
################

In [9]:
bt_tpsl_grid(df)

Pour un TP de   0.1  et un SL de   0.1 
fermeture short en l'air
Winner Ratio : 0.0 %
Profit Factor :  0.0 % 
Nombre de trades : 1
Nombre de posistions non fermées 0
Captital initial : 250000
Taille des positions : 50000
Gain :  -3939.249999999994 
Capital Final : 246060.75

________________________________________________________________________________________

Pour un TP de   0.1  et un SL de   0.09 
fermeture short en l'air
Winner Ratio : 0.0 %
Profit Factor :  0.0 % 
Nombre de trades : 1
Nombre de posistions non fermées 0
Captital initial : 250000
Taille des positions : 50000
Gain :  -3939.249999999994 
Capital Final : 246060.75

________________________________________________________________________________________

Pour un TP de   0.1  et un SL de   0.08 
fermeture short en l'air
Winner Ratio : 50.0 %
Profit Factor :  0.12 % 
Nombre de trades : 2
Nombre de posistions non fermées 0
Captital initial : 250000
Taille des positions : 50000
Gain :  -3939.2499999999945 
Capital Final 

()